In [ ]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
from cartopy import crs as ccrs, feature as cfeature
import numpy as np
from climada.hazard import TCTracks
from climada.hazard import Centroids, TropCyclone
import climada.util.coordinates as u_coord
from tc_analysis import windspeed_analysis
from tc_analysis import plot as tcplot
import tc_analysis
import pandas as pd
from shapely.geometry import Point, LineString, Polygon
from roms_func import forcing, input_control, validation, postprocessing
from roms_func import plot as rplot
import subprocess
import os

In [ ]:
year_start = 1980
year_end = 2020
basin = 'NA'
fp_tg = 'h765a.nc'
fp_grd = './roms_grd.nc'
proj = 'lousiana'

In [ ]:
os.chdir(proj)

In [ ]:
# Open tide gauge file and get time bounds
ds_tg = xr.open_dataset(fp_tg)
year_start = max( pd.to_datetime( ds_tg.time.min().values ).year, year_start+1 )
year_end = min( pd.to_datetime( ds_tg.time.max().values ).year, year_end-1 )

In [ ]:
# Read tracks between start and end years
tracks = TCTracks.from_ibtracs_netcdf(basin=basin, year_range=[year_start,year_end])

In [ ]:
# Open grid file and get grid polygon
ds_grd = xr.open_dataset( fp_grd )
grid_poly = forcing.get_grid_poly( ds_grd.lon_rho, ds_grd.lat_rho)

# Subset tracks in grid domain
tracks.data = tc_analysis.subset_tracks_in_poly(tracks.data, grid_poly)

# Filter out storms that don't pass close to central tide gauge
min_dist = 1.5
tg_lon = ds_tg.lon.values[0]
if tg_lon > 180:
    tg_lon = tg_lon-360
tg_lat = ds_tg.lat.values[0]
tg_poly = Point( [tg_lon, tg_lat] ).buffer(min_dist)
tracks.data = tc_analysis.subset_tracks_in_poly(tracks.data, tg_poly)

# Filter out weak storms
tracks.data = tc_analysis.filter_tracks_by_intensity( tracks.data, min_intensity = 64 )

In [ ]:
n_storms = len(tracks.data)
n_storms

In [ ]:
val = []
reject = []
year_list = np.arange(year_start, year_end+1)
subprocess.run('rm -rf maxima/*', shell=True)
os.remove('roms_his.nc')

for yy in year_list:

    # Extract tracks for this year
    tracks_yy = tc_analysis.subset_tracks_in_year( tracks, yy )

    # If empty, save empty envelope and continue onto the next year
    n_storms_year = len(tracks_yy.data)
    if n_storms_year == 0:
        print(f'Year {yy} / {year_end} --> No tracks found')
        ds_zmax = postprocessing.make_zero_surge_envelope( )
        ds_zmax['year'] = yy
        ds_zmax.to_netcdf( f'./maxima/zmax_y{yy}.nc') 
        continue

    z_envelopes = []
    
    for ii in range(n_storms_year):
        
        storm = tracks_yy.data[ii]
        print(f'Year {yy} / {year_end} --> {ii+1} / {n_storms_year} ---> {storm.sid}')
    
        # Make forcing for this storm
        subprocess.run(f'python ./make_forcing_ibtracs.py -sid {storm.sid} -freq .25', shell=True,)
    
        # Check there are enough obs to warrant running the model
        ndata_ha, ndata_frac = validation.check_obs_from_file( fp_tg )
        if ndata_frac < .25 or ndata_ha < 30*24:
            print(f'Not enough obs. REJECT. frac = {ndata_frac}, ha = {ndata_ha}')
            continue
    
        # Don't reject but do flag data
        rejii = False
        if ndata_ha < 3*30*24:
            rejii = True
        if ndata_frac < 0.75:
            rejii = True
        reject.append(rejii)
        
        # Make input control file
        input_control.make_infile_from_files( )
    
        # Run model
        subprocess.run( f'mpirun -np 4 --oversubscribe romsM roms.in > log.txt', shell=True,)
    
        # Get validation stats
        ds_v, obs, mod = validation.validate_from_file(fp_tg = fp_tg, sid=storm.sid)
        val.append(ds_v)
    
        # Make plots
        f,a = rplot.plot_validation_timseries( mod, obs )
        f.savefig( f'./plots/valts_{storm.sid}' )
        plt.close('all')
    
        # Make annual max
        z_envelopes.append(postprocessing.calculate_surge_envelope())

    ds_zmax = xr.concat(z_envelopes, dim='storm').max(dim='storm').drop('h')
    ds_zmax['year'] = yy
    ds_zmax.to_netcdf( f'./maxima/zmax_y{yy}.nc') 
    
df_val = pd.concat(val)
df_val['reject'] = reject

In [ ]:
df_val.to_csv('validation.csv')

In [ ]:
df_val